In [2]:
%conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /Users/jingzhao/anaconda3

  added / updated specs:
    - pandas


The following packages will be UPDATED:

  pandas                              2.0.3-py311hdb55bb0_0 --> 2.1.4-py311hdb55bb0_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [3]:
%conda install statsmodels

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [4]:
%conda install scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [55]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from statsmodels.api import OLS
import itertools

In [84]:
# Import data
financial_data = pd.read_csv("/Users/jingzhao/Desktop/FE PW1/Data/Financial Data.CSV")
industry_data = pd.read_csv("/Users/jingzhao/Desktop/FE PW1/Data/17 Industry Portfolios.CSV")
financial_data['Date'] = pd.to_datetime(financial_data['Date'], format='%Y%m')
industry_data['Date'] = pd.to_datetime(industry_data['Date'], format='%Y%m')
merged_data = pd.merge(financial_data, industry_data, on='Date')

In [101]:
# HML: Choose Best Regressor

targets = ['SV', 'BV', 'SG', 'BG']
regressors = industry_data.columns[1:]
feature_importances = {}

# Modeling and feature importance extraction
for target in targets:
    X = merged_data[regressors]
    y = merged_data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Evaluating the model
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Storing feature importances
    importances = rf.feature_importances_
    feature_importances[target] = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    print(f"Model for {target}:")
    print(f"Mean Squared Error: {mse}")
    print("Top 5 Important Features:")
    print(feature_importances[target].head(5))
    print("\n")

feature_importances

Model for SV:
Mean Squared Error: 14.421394358166129
Top 5 Important Features:
Trans    0.358771
Cnstr    0.152311
Durbl    0.095938
Steel    0.075731
Clths    0.067231
dtype: float64


Model for BV:
Mean Squared Error: 5.431219203378785
Top 5 Important Features:
Finan    0.338756
Trans    0.281856
Steel    0.084791
FabPr    0.078939
Other    0.037873
dtype: float64


Model for SG:
Mean Squared Error: 11.175908652000558
Top 5 Important Features:
Other    0.492491
Machn    0.099339
Trans    0.085267
Cnstr    0.081539
Clths    0.051165
dtype: float64


Model for BG:
Mean Squared Error: 1.8069052822806653
Top 5 Important Features:
Other    0.634174
Machn    0.161844
Rtail    0.032322
Finan    0.027878
Chems    0.023263
dtype: float64




{'SV': Trans    0.358771
 Cnstr    0.152311
 Durbl    0.095938
 Steel    0.075731
 Clths    0.067231
 Other    0.061022
 Finan    0.044507
 Rtail    0.039967
 FabPr    0.019101
 Mines    0.016708
 Utils    0.012997
 Cars     0.012313
 Food     0.011588
 Oil      0.011188
 Cnsum    0.007738
 Machn    0.007199
 Chems    0.005690
 dtype: float64,
 'BV': Finan    0.338756
 Trans    0.281856
 Steel    0.084791
 FabPr    0.078939
 Other    0.037873
 Durbl    0.024430
 Cnstr    0.022717
 Machn    0.018143
 Chems    0.017380
 Oil      0.016545
 Clths    0.015393
 Utils    0.014138
 Cnsum    0.013097
 Food     0.010343
 Cars     0.010321
 Rtail    0.009142
 Mines    0.006138
 dtype: float64,
 'SG': Other    0.492491
 Machn    0.099339
 Trans    0.085267
 Cnstr    0.081539
 Clths    0.051165
 Durbl    0.036151
 FabPr    0.027351
 Finan    0.021493
 Rtail    0.019101
 Steel    0.016451
 Cars     0.016068
 Mines    0.012472
 Food     0.010310
 Chems    0.009199
 Utils    0.008663
 Oil      0.00671

In [86]:
# HML: Variable Regression
def run_ols(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.summary()

ols_results = {}
top_features = {target: features.drop('Other').head(4) for target, features in feature_importances.items()}

for target in targets:
    print(f"OLS Regression for {target}:")
    selected_features = top_features[target].index
    X = merged_data[selected_features]
    y = merged_data[target]
    ols_results[target] = run_ols(X, y)
    print(ols_results[target])
    print("\n")

OLS Regression for SV:
                            OLS Regression Results                            
Dep. Variable:                     SV   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     1490.
Date:                Fri, 05 Jan 2024   Prob (F-statistic):               0.00
Time:                        20:51:51   Log-Likelihood:                -3030.0
No. Observations:                1163   AIC:                             6070.
Df Residuals:                    1158   BIC:                             6095.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3965      0.

In [91]:
# HML Regression & Accuracy
def rolling_regression_with_mse(data, train_years, hold_years):
    train_months = train_years * 12
    hold_months = hold_years * 12
    total_months = train_months + hold_months

    mse_values = []
    start_period = 0

    # Loop through the data with the specified rolling window
    while start_period + total_months <= len(data):
        # Split the data into training and holding sets
        train_data = data.iloc[start_period:start_period + train_months]
        hold_data = data.iloc[start_period + train_months:start_period + total_months]

        # Train the model
        X_train = train_data[['SV', 'BV', 'SG', 'BG']]
        y_train = train_data['HML']
        model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

        # Predict on the holdout data
        X_hold = hold_data[['SV', 'BV', 'SG', 'BG']]
        y_hold_actual = hold_data['HML']
        y_hold_pred = model.predict(sm.add_constant(X_hold))

        # Calculate and store MSE
        mse = mean_squared_error(y_hold_actual, y_hold_pred)
        mse_values.append(mse)

        # Move to the next window
        start_period += hold_months

    return np.mean(mse_values)

# Calculate MSE for each training window length
mse_5y = rolling_regression_with_mse(merged_data, 5, 5)
mse_10y = rolling_regression_with_mse(merged_data, 10, 5)
mse_20y = rolling_regression_with_mse(merged_data, 20, 5)

print("mse_5y =", mse_5y), print("mse_10y =", mse_10y), print("mse_20y =", mse_20y)

mse_5y = 9.544718888764207e-06
mse_10y = 8.974625317379676e-06
mse_20y = 8.817379562779719e-06


(None, None, None)

In [105]:
# HML Regression & Accuracy
def rolling_regression_with_adjusted_r_squared(data, train_years, hold_years):
    train_months = train_years * 12
    hold_months = hold_years * 12
    total_months = train_months + hold_months

    mse_values = []
    adjusted_r_squared_values = []
    start_period = 0

    while start_period + total_months <= len(data):
        # Split the data into training and holding sets
        train_data = data.iloc[start_period:start_period + train_months]
        hold_data = data.iloc[start_period + train_months:start_period + total_months]

        # Train the model
        X_train = train_data[['SV', 'BV', 'SG', 'BG']]
        y_train = train_data['HML']
        model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

        # Predict on the holdout data
        X_hold = hold_data[['SV', 'BV', 'SG', 'BG']]
        y_hold_actual = hold_data['HML']
        y_hold_pred = model.predict(sm.add_constant(X_hold))

        # Calculate and store MSE and Adjusted R-squared
        mse = mean_squared_error(y_hold_actual, y_hold_pred)
        mse_values.append(mse)
        adjusted_r_squared_values.append(model.rsquared_adj)

        # Move to the next window
        start_period += hold_months

    average_mse = np.mean(mse_values)
    average_adjusted_r_squared = np.mean(adjusted_r_squared_values)
    return average_mse, average_adjusted_r_squared

# Perform rolling window regression for each training window length
mse_5y, adj_r_squared_5y = rolling_regression_with_adjusted_r_squared(merged_data, 5, 5)
mse_10y, adj_r_squared_10y = rolling_regression_with_adjusted_r_squared(merged_data, 10, 5)
mse_20y, adj_r_squared_20y = rolling_regression_with_adjusted_r_squared(merged_data, 20, 5)

mse_5y, adj_r_squared_5y, mse_10y, adj_r_squared_10y, mse_20y, adj_r_squared_20y
print("mse_5y =", mse_5y), print("mse_10y =", mse_10y), print("mse_20y =", mse_20y)
print("adj_r_squared_5y =", adj_r_squared_5y), print("adj_r_squared_10y =", adj_r_squared_10y), print("adj_r_squared_20y =", adj_r_squared_20y)

mse_5y = 9.544718888764207e-06
mse_10y = 8.974625317379676e-06
mse_20y = 8.817379562779719e-06
adj_r_squared_5y = 0.9999985511375002
adj_r_squared_10y = 0.9999987733325719
adj_r_squared_20y = 0.9999988605139928


(None, None, None)

In [98]:
# MON: Best Regressors
targets = ['SH', 'BH', 'SL', 'BL']
regressors = industry_data.columns[1:]

feature_importances = {}

# Modeling and feature importance extraction
for target in targets:
    X = merged_data[regressors]
    y = merged_data[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)

    # Evaluating the model
    y_pred = rf.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Storing feature importances
    importances = rf.feature_importances_
    feature_importances[target] = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    print(f"Model for {target}:")
    print(f"Mean Squared Error: {mse}")
    print("Top 5 Important Features:")
    print(feature_importances[target].head(5))
    print("\n")

feature_importances

Model for SH:
Mean Squared Error: 17.55622752781117
Top 5 Important Features:
Other    0.378354
Cnstr    0.155564
Trans    0.091979
Machn    0.074735
Durbl    0.056284
dtype: float64


Model for BH:
Mean Squared Error: 4.277504814978542
Top 5 Important Features:
Other    0.651636
Machn    0.060819
Cnstr    0.045426
Oil      0.031122
Steel    0.026370
dtype: float64


Model for SL:
Mean Squared Error: 18.477486118626594
Top 5 Important Features:
Other    0.287929
Trans    0.230841
Clths    0.099695
Finan    0.076034
Durbl    0.054691
dtype: float64


Model for BL:
Mean Squared Error: 11.782296854377682
Top 5 Important Features:
Finan    0.271912
Other    0.227828
Trans    0.089599
Steel    0.081705
FabPr    0.080684
dtype: float64




{'SH': Other    0.378354
 Cnstr    0.155564
 Trans    0.091979
 Machn    0.074735
 Durbl    0.056284
 Steel    0.051744
 Clths    0.041034
 Finan    0.032997
 FabPr    0.020836
 Rtail    0.017296
 Mines    0.016198
 Cars     0.013414
 Food     0.012125
 Oil      0.010462
 Utils    0.010339
 Chems    0.009450
 Cnsum    0.007189
 dtype: float64,
 'BH': Other    0.651636
 Machn    0.060819
 Cnstr    0.045426
 Oil      0.031122
 Steel    0.026370
 FabPr    0.024777
 Trans    0.024553
 Durbl    0.023606
 Food     0.020833
 Finan    0.016360
 Rtail    0.014878
 Utils    0.012777
 Cnsum    0.011702
 Chems    0.011687
 Mines    0.008699
 Clths    0.007423
 Cars     0.007330
 dtype: float64,
 'SL': Other    0.287929
 Trans    0.230841
 Clths    0.099695
 Finan    0.076034
 Durbl    0.054691
 Cnstr    0.054456
 Rtail    0.036567
 Steel    0.030371
 FabPr    0.024348
 Machn    0.023794
 Cars     0.019273
 Mines    0.018579
 Utils    0.011183
 Oil      0.010298
 Chems    0.008090
 Food     0.00738

In [99]:
# MOM: Variable Regression
def run_ols(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model.summary()

ols_results = {}

top_features = {target: features.drop('Other').head(4) for target, features in feature_importances.items()}

for target in targets:
    print(f"OLS Regression for {target}:")
    selected_features = top_features[target].index
    X = merged_data[selected_features]
    y = merged_data[target]
    ols_results[target] = run_ols(X, y)
    print(ols_results[target])
    print("\n")

OLS Regression for SH:
                            OLS Regression Results                            
Dep. Variable:                     SH   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     1122.
Date:                Fri, 05 Jan 2024   Prob (F-statistic):               0.00
Time:                        21:33:27   Log-Likelihood:                -3027.5
No. Observations:                1163   AIC:                             6065.
Df Residuals:                    1158   BIC:                             6090.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5990      0.

In [106]:
# MOM Regression & Accuracy
def rolling_regression_with_adjusted_r_squared(data, train_years, hold_years):
    train_months = train_years * 12
    hold_months = hold_years * 12
    total_months = train_months + hold_months

    mse_values = []
    adjusted_r_squared_values = []
    start_period = 0

    while start_period + total_months <= len(data):
        # Split the data into training and holding sets
        train_data = data.iloc[start_period:start_period + train_months]
        hold_data = data.iloc[start_period + train_months:start_period + total_months]

        # Train the model
        X_train = train_data[['SH', 'BH', 'SL', 'BL']]
        y_train = train_data['MOM']
        model = sm.OLS(y_train, sm.add_constant(X_train)).fit()

        # Predict on the holdout data
        X_hold = hold_data[['SH', 'BH', 'SL', 'BL']]
        y_hold_actual = hold_data['MOM']
        y_hold_pred = model.predict(sm.add_constant(X_hold))

        # Calculate and store MSE and Adjusted R-squared
        mse = mean_squared_error(y_hold_actual, y_hold_pred)
        mse_values.append(mse)
        adjusted_r_squared_values.append(model.rsquared_adj)

        # Move to the next window
        start_period += hold_months

    average_mse = np.mean(mse_values)
    average_adjusted_r_squared = np.mean(adjusted_r_squared_values)
    return average_mse, average_adjusted_r_squared

# Perform rolling window regression for each training window length
mse_5y, adj_r_squared_5y = rolling_regression_with_adjusted_r_squared(merged_data, 5, 5)
mse_10y, adj_r_squared_10y = rolling_regression_with_adjusted_r_squared(merged_data, 10, 5)
mse_20y, adj_r_squared_20y = rolling_regression_with_adjusted_r_squared(merged_data, 20, 5)

mse_5y, adj_r_squared_5y, mse_10y, adj_r_squared_10y, mse_20y, adj_r_squared_20y
print("mse_5y =", mse_5y), print("mse_10y =", mse_10y), print("mse_20y =", mse_20y)
print("adj_r_squared_5y =", adj_r_squared_5y), print("adj_r_squared_10y =", adj_r_squared_10y), print("adj_r_squared_20y =", adj_r_squared_20y)

mse_5y = 1.7985837549286397e-05
mse_10y = 1.7138158869586988e-05
mse_20y = 1.663063943183368e-05
adj_r_squared_5y = 0.9999984083129649
adj_r_squared_10y = 0.999998636171096
adj_r_squared_20y = 0.9999987370515305


(None, None, None)